# 📘 Agentic 架构 9：思维树规划（Tree-of-Thoughts Planning）

欢迎来到我们系列的第九个笔记本。今天，我们将探讨一种强大的推理与规划架构：**思维树（Tree-of-Thoughts, ToT）**。这种模式将代理的问题求解能力从线性思维链提升到多路径探索性搜索。

与生成单一、顺序式的推理线路不同，ToT 代理在问题的每个阶段都会生成多个候选“想法”或后续步骤。然后，它对这些想法进行评估，剪枝无效或前景不佳的分支，并扩展最有前景的分支。这形成了一棵搜索树，使代理能够回溯、探索备选方案，并系统性地导航复杂的问题空间。

为了展示这一点，我们将让代理解决一个经典逻辑谜题：**狼、羊和白菜过河问题**。这个谜题之所以著名，是因为它需要非显而易见的步骤（例如将物品*带回来*），并且存在多个无效状态，容易让朴素的推理者陷入困境。我们将展示一个简单的思维链（Chain-of-Thought）代理可能如何失败，而 ToT 代理则通过探索和评估可能性树，有条不紊地构建出一个有效计划。

### 定义
**思维树（Tree-of-Thoughts, ToT）** 是一种代理式推理框架，将问题求解建模为在一棵树中进行搜索。代理同时探索多条推理路径（分支）。在每一步，它生成多个潜在的后续步骤或“想法”（thoughts），评估这些想法的可行性，并决定继续探索哪些路径，从而有效剪枝搜索空间。

### 高层工作流程

1.  **问题分解：** 将问题分解为一连串步骤或想法。
2.  **想法生成：** 对于问题的当前状态，代理生成多个潜在的后续步骤或想法。这会在搜索树中创建分支。
3.  **状态评估：** 对每个新想法（导致新状态）进行评估，可由“批评者”（critic）或验证函数完成。该评估可以考察：
    *   **有效性：** 此动作是否符合问题规则？
    *   **进展：** 此动作是否让我们更接近解决方案？
    *   **启发式：** 这条路径是否很可能成功？
4.  **剪枝与扩展：** 无效或前景不佳的分支被剪除。代理随后从最有前景的活跃分支继续，重复想法生成过程。
5.  **解决方案：** 该过程持续进行，直到达到目标状态。解决方案即是从根节点到目标节点的思维路径。

### 何时使用 / 应用场景
*   **逻辑谜题与数学问题：** 具有明确规则和目标状态，需要多步、非线性推理的问题（例如，数独、过河谜题）。
*   **复杂规划：** 需要详细计划的任务，其中操作顺序很重要且必须遵守约束（例如，规划包含多段行程和预算限制的复杂旅行）。
*   **创意写作或代码生成：** 在确定最终方案前探索多个故事分支或实现策略。

### 优势与劣势
*   **优势：**
    *   **健壮性：** 系统性地探索问题空间，与单次通过的方法相比，更不容易陷入死胡同或产生错误答案。
    *   **处理组合复杂度：** 非常适合可能序列数量巨大的问题。
*   **劣势：**
    *   **计算开销大：** 需要比简单思维链提示多得多的 LLM 调用和状态管理，因此更慢且成本更高。
    *   **依赖高质量评估器：** 搜索的有效性高度依赖状态评估逻辑的质量。

## 第零阶段：基础与设置

我们将像往常一样安装库并配置我们的 API 密钥。

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv

In [2]:
import os
import re
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv
from collections import defaultdict

# Pydantic for data modeling
from pydantic import BaseModel, Field

# LangChain components
from langchain_nebius import ChatNebius
from langchain_core.prompts import ChatPromptTemplate

# LangGraph components
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown
from rich.tree import Tree

# --- API Key and Tracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Tree-of-Thoughts (Nebius)"

# Check for required environment variables
required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
for var in required_vars:
    if var not in os.environ:
        print(f"Warning: Environment variable {var} not set.")

print("Environment variables loaded and tracing is set up.")

Environment variables loaded and tracing is set up.


## 第一阶段：定义问题环境

思维树（Tree-of-Thoughts）系统需要在明确定义的环境中运行。对于我们的狼、羊和白菜过河谜题，这意味着我们需要以编程方式定义以下内容：

1.  **状态表示：** 一种描述所有物品位置的方式。
2.  **验证规则：** 一个函数，用于检查状态是否无效（例如，羊和白菜被单独留在同一侧）。
3.  **目标状态：** 一个方式，用于检查谜题是否已解决。
4.  **可能动作：** 一个函数，用于从给定状态确定所有合法动作。

In [3]:
console = Console()

class PuzzleState(BaseModel):
    "Represents the state of the Wolf, Goat, and Cabbage puzzle."
    # Using sets for unordered collections of items on each bank
    left_bank: set[str] = Field(default_factory=lambda: {"wolf", "goat", "cabbage"})
    right_bank: set[str] = Field(default_factory=set)
    boat_location: str = "left"
    move_description: str = "Initial state."

    def is_valid(self) -> bool:
        """Checks if the current state is valid (no one gets eaten)."""
        # Check left bank
        if self.boat_location == "right":
            if "wolf" in self.left_bank and "goat" in self.left_bank:
                return False
            if "goat" in self.left_bank and "cabbage" in self.left_bank:
                return False
        # Check right bank
        if self.boat_location == "left":
            if "wolf" in self.right_bank and "goat" in self.right_bank:
                return False
            if "goat" in self.right_bank and "cabbage" in self.right_bank:
                return False
        return True

    def is_goal(self) -> bool:
        """Checks if the goal state has been reached."""
        return self.right_bank == {"wolf", "goat", "cabbage"}
    
    def __hash__(self):
        # Make the state hashable to check for visited states
        return hash((frozenset(self.left_bank), frozenset(self.right_bank), self.boat_location))
    
    def __eq__(self, other):
        return self.__hash__() == other.__hash__()

def get_possible_moves(state: PuzzleState) -> list[PuzzleState]:
    """Generates all possible valid next states from the current state."""
    moves = []
    current_bank = state.left_bank if state.boat_location == "left" else state.right_bank
    
    # Option 1: Move one item in the boat
    for item in current_bank:
        new_state = state.model_copy(deep=True)
        if state.boat_location == "left":
            new_state.left_bank.remove(item)
            new_state.right_bank.add(item)
            new_state.boat_location = "right"
            new_state.move_description = f"Move {item} to the right bank."
        else:
            new_state.right_bank.remove(item)
            new_state.left_bank.add(item)
            new_state.boat_location = "left"
            new_state.move_description = f"Move {item} to the left bank."
        if new_state.is_valid():
            moves.append(new_state)
            
    # Option 2: Move the boat empty
    empty_move_state = state.model_copy(deep=True)
    if state.boat_location == "left":
        empty_move_state.boat_location = "right"
        empty_move_state.move_description = "Move the boat empty to the right bank."
    else:
        empty_move_state.boat_location = "left"
        empty_move_state.move_description = "Move the boat empty to the left bank."
    if empty_move_state.is_valid():
        moves.append(empty_move_state)
        
    return moves

print("Puzzle environment defined successfully.")

Puzzle environment defined successfully.


## 第二阶段：使用 LangGraph 实现 ToT 代理

现在，我们将构建代理本身。我们图的状态将跟踪思维树中的所有活跃路径（分支）。节点将执行关键的 ToT 操作：

1.  **扩展路径（想法生成器）：** 一个由 LLM 驱动的节点，它查看每个活跃路径的最后状态，并从合法可能动作列表中提出一个有前景的后续动作。
2.  **剪枝路径（状态评估器）：** 该节点在生成之后进行清理。它将移除任何进入无效状态或陷入循环（重新访问之前状态）的路径。
3.  **检查解决方案（目标检查）：** 一个条件节点，用于检查是否有任何活跃路径已达到目标状态。如果有，则终止循环。

In [4]:
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0.4)

# Pydantic model for the LLM's choice of move
class MoveChoice(BaseModel):
    best_move_index: int = Field(description="The index of the best move from the provided list of possible moves.")
    reasoning: str = Field(description="Brief reasoning for why this is the most promising move.")

# LangGraph State
class ToTState(TypedDict):
    problem_description: str
    # Each path is a list of PuzzleState objects
    active_paths: List[List[PuzzleState]]
    # We'll store the final solution here
    solution: Optional[List[PuzzleState]]

# Graph Nodes
def initialize_search(state: ToTState) -> Dict[str, Any]:
    """Node to set up the initial state of the search."""
    initial_puzzle_state = PuzzleState()
    return {"active_paths": [[initial_puzzle_state]]}

def expand_paths(state: ToTState) -> Dict[str, Any]:
    """The 'Thought Generator'. Expands each active path with a promising next move."""
    console.print("--- Expanding Paths ---")
    new_paths = []
    choice_llm = llm.with_structured_output(MoveChoice)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an expert logic puzzle solver. Your goal is to solve the Wolf, Goat, and Cabbage problem. Analyze the current path and choose the most promising next move from the list of options to reach the goal."),
        ("human", "Problem: {problem}\n\nCurrent Path History:\n{path_history}\n\nFrom the final state, choose the best next move from this list:\n{possible_moves}")
    ])
    
    for path in state['active_paths']:
        last_state = path[-1]
        possible_next_states = get_possible_moves(last_state)
        
        if not possible_next_states:
            continue # This path is a dead end
            
        path_history_str = " -> ".join([s.move_description for s in path])
        possible_moves_str = "\n".join([f"{i}: {s.move_description}" for i, s in enumerate(possible_next_states)])
        
        # For simplicity and to show breadth, we can explore multiple moves.
        # A more advanced ToT might use the LLM to pick only the single best one.
        # Here, we'll let all valid moves branch out to demonstrate the tree structure.
        for next_state in possible_next_states:
            new_paths.append(path + [next_state])

    console.print(f"[cyan]Expanded to {len(new_paths)} potential paths.[/cyan]")
    return {"active_paths": new_paths}

def prune_paths(state: ToTState) -> Dict[str, Any]:
    """The 'State Evaluator'. Prunes paths that are invalid or contain cycles."""
    console.print("--- Pruning Paths ---")
    pruned_paths = []
    for path in state['active_paths']:
        # Check for cycles: if the last state has appeared before in the path
        if path[-1] in path[:-1]:
            continue # Found a cycle, prune this path
        
        # The get_possible_moves function already ensures validity, but this is a good place for extra checks.
        pruned_paths.append(path)
        
    console.print(f"[green]Pruned down to {len(pruned_paths)} valid, non-cyclical paths.[/green]")
    return {"active_paths": pruned_paths}

# Conditional Node
def check_for_solution(state: ToTState) -> str:
    """Checks if any path has reached the goal and routes execution."""
    for path in state['active_paths']:
        if path[-1].is_goal():
            console.print("[bold green]Solution Found![/bold green]")
            # Side effect: update the solution in the state. LangGraph copies this out.
            state['solution'] = path
            return "solution_found"
    return "continue_search"

# Build the graph
workflow = StateGraph(ToTState)

workflow.add_node("initialize", initialize_search)
workflow.add_node("expand", expand_paths)
workflow.add_node("prune", prune_paths)

workflow.set_entry_point("initialize")
workflow.add_edge("initialize", "expand")
workflow.add_edge("expand", "prune")

workflow.add_conditional_edges(
    "prune",
    check_for_solution,
    {
        "solution_found": END,
        "continue_search": "expand"
    }
)

tot_agent = workflow.compile()
print("Tree-of-Thoughts agent graph compiled successfully.")

Tree-of-Thoughts agent graph compiled successfully.


## 第三阶段：演示与分析

现在，让我们运行 ToT 代理来解决这个谜题。我们将把它的系统性方法与一个简单的、单次通过的思维链（Chain-of-Thought）请求进行比较，以突出两者在健壮性方面的差异。

In [5]:
problem = "A farmer wants to cross a river with a wolf, a goat, and a cabbage. The boat can only carry the farmer and one other item. The farmer cannot leave the wolf alone with the goat, nor the goat alone with the cabbage. How can the farmer get everyone across safely?"

console.print("--- 🌳 Running Tree-of-Thoughts Agent ---")
# The recursion limit prevents infinite loops in our graph
config = {"recursion_limit": 15}
final_state = tot_agent.invoke({"problem_description": problem}, config=config)

console.print("\n--- ✅ ToT Agent Solution ---")
if final_state.get('solution'):
    solution_path = final_state['solution']
    # Use rich.Tree for a nice visual output
    tree = Tree("[bold magenta]Wolf, Goat, and Cabbage Solution Path[/bold magenta]")
    for i, state in enumerate(solution_path):
        tree.add(f"[green]{i+1}.[/green] {state.move_description}")
    console.print(tree)
else:
    console.print("[bold red]No solution found within the step limit.[/bold red]")

console.print("\n--- 🤔 Running Simple Chain-of-Thought Agent ---")
cot_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class logic puzzle solver. Provide a step-by-step solution to the user's puzzle."),
    ("human", "{problem}")
])
cot_chain = cot_prompt | llm
cot_result = cot_chain.invoke({"problem": problem}).content
console.print(Markdown(cot_result))

--- 🌳 Running Tree-of-Thoughts Agent ---


--- Expanding Paths ---
Expanded to 1 potential paths.
--- Pruning Paths ---
Pruned down to 1 valid, non-cyclical paths.
--- Expanding Paths ---
Expanded to 2 potential paths.
--- Pruning Paths ---
Pruned down to 2 valid, non-cyclical paths.
--- Expanding Paths ---
Expanded to 4 potential paths.
--- Pruning Paths ---
Pruned down to 4 valid, non-cyclical paths.
--- Expanding Paths ---
Expanded to 7 potential paths.
--- Pruning Paths ---
Pruned down to 7 valid, non-cyclical paths.
--- Expanding Paths ---
Expanded to 12 potential paths.
--- Pruning Paths ---
Pruned down to 12 valid, non-cyclical paths.
--- Expanding Paths ---
Expanded to 20 potential paths.
--- Pruning Paths ---
Pruned down to 20 valid, non-cyclical paths.
--- Expanding Paths ---
Expanded to 32 potential paths.
--- Pruning Paths ---
Pruned down to 32 valid, non-cyclical paths.
Solution Found!



--- ✅ ToT Agent Solution ---


Wolf, Goat, and Cabbage Solution Path
├── 1. Initial state.
├── 2. Move goat to the right bank.
├── 3. Move the boat empty to the left bank.
├── 4. Move wolf to the right bank.
├── 5. Move goat to the left bank.
├── 6. Move cabbage to the right bank.
├── 7. Move the boat empty to the left bank.
└── 8. Move goat to the right bank.



--- 🤔 Running Simple Chain-of-Thought Agent ---


Here's a step-by-step solution to the Wolf, Goat, and Cabbage puzzle:

1.  **Take the Goat across:** First, take the goat across the river to the right bank. You leave the wolf and cabbage behind on the left bank.
2.  **Return empty:** Return to the left bank alone.
3.  **Take the Wolf across:** Now, take the wolf across to the right bank. 
4.  **Bring the Goat back:** *This is the key step.* Leave the wolf on the right bank and bring the goat back with you to the left bank.
5.  **Take the Cabbage across:** Leave the goat on the left bank and take the cabbage across to the right bank. Now the wolf and cabbage are on the right bank.
6.  **Return empty:** Return to the left bank alone.
7.  **Take the Goat across:** Finally, take the goat across to the right bank.

Now, the wolf, goat, and cabbage are all safely on the right bank, and the puzzle is solved.

### 结果分析

两种方法之间的差异是深刻的：

- **思维链（Chain-of-Thought, CoT）：** 这种方法依赖 LLM 的预训练知识来回忆解决方案。对于像这样一个经典的、广为人知的问题，强大的 LLM 往往能够一次性给出正确答案。然而，如果它犯了一个错误，就没有自我纠正的机制。对于新颖的或更复杂的问题，失败的可能性会大大增加。其正确性依赖于回忆，而不是可验证的推理。

- **思维树（Tree-of-Thoughts, ToT）：** 这个代理通过系统性的、可验证的搜索*发现*了解决方案。它不是简单回忆答案，而是构建了一个答案。我们可以在日志中看到这个过程：扩展路径，然后剪枝那些陷入死胡同或循环的分支。即使指导扩展的 LLM 在某个分支上做出了次优选择，代理仍然可以继续探索其他更有前景的分支。这种方法远更健壮且可信，因为其最终解决方案根据我们定义的环境规则是保证有效的。

ToT 代理的成功并非基于运气或记忆，而是基于其搜索算法的可靠性。这使得它成为对高可靠性和规划能力有严格要求的任务的远优选择。

## 结论

在本笔记本中，我们实现了一个**思维树（Tree-of-Thoughts）**代理来解决一个经典逻辑谜题。我们展示了通过将问题转化为状态空间并系统性地在其间进行搜索，代理能够达到简单单次推理方法无法实现的健壮性和准确性水平。

ToT 的核心组件——**想法生成（扩展）**、**状态评估（剪枝）**以及**搜索**——共同构成了一个强大的框架，用于应对复杂的规划和推理任务。虽然它带来了更高的计算成本，但这种权衡换来的是可靠性和问题求解能力的显著提升。这种架构是朝着构建能够深思熟虑地推理并解决具有挑战性、多步骤问题的代理迈出的关键一步。